In [2]:
import pandas as pd
import uuid
from datetime import datetime
import numpy as np
from grakn.client import GraknClient
from tqdm import tqdm
tqdm.pandas()

In [3]:
df=pd.read_excel(r"C:\Users\SRIKANTH\Desktop\OpenCTI_Data.xlsx",sheet_name='Vulnerability')

In [4]:
def  insert_general_info(df,ent_type,create):
    if create:
        for index,i in df.iterrows():
            df.internal_id_key[index]=str(uuid.uuid4())
            df.stix_id_key[index]=ent_type+'--'+str(uuid.uuid4())
            df.stix_label[index]='NA'
            df.entity_type[index]=ent_type
            df.created=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.created_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.created_at_day=(datetime.now()).strftime("%Y-%m-%d")
            df.created_at_month=(datetime.now()).strftime("%Y-%m")
            df.created_at_year=(datetime.now()).strftime("%Y")
            df.modified=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.updated_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.revoked=False
            df.object_status='NA'
    else:
            df.modified=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.updated_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
    return df
    

In [127]:
result=insert_general_info(df=df,ent_type='Vulnerability',create=True)
result.fillna('NA',inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is t

In [1]:
result['tool_version']

NameError: name 'result' is not defined

In [129]:
def insert_one_vulnerability (df,session):
    #if df.first_seen != 'NA':
     #   f=df.first_seen.strftime("%Y-%m-%dT%H:%M:%S")
     #   f_m=df.first_seen.strftime("%Y-%m")
     #   f_d=df.first_seen.strftime("%Y-%m-%d")
    #elif df.first_seen_month != 'NA':
    #    f_m=df.first_seen_month
    #elif df.first_seen_day != 'NA':
    #    f_d=df.first_seen_day
    #else:
    #    f_d=None
    #    f_m=None
    #    f=None
    df['description']=df['description'].replace('\"','').strip()
    aliases = list()
    if df['alias'] != 'NA':            
        aliases=df.alias.split('\n')

    graql_match_query = f"match $i isa Vulnerability , " \
                        f"has name \"{df['name']}__\"; " \
                        f"get;"
    # print(graql_match_query)
    with session.transaction().read() as read_transaction:
        # read_transaction.query(graql_match_query)
        answer_iterator = None
        answer_iterator = read_transaction.query(graql_match_query, infer=True)
        data = answer_iterator.collect_concepts()
                
        if len(data) > 0:
            # Update data.
            # Columns to be updated: object_status, revoked, updated_at, modified, description, alias
            # print("Update intrusion-set...")
            print("Update: " + df['name'])
            graql_attribute_delete_query = f"match $i isa Vulnerability , " \
                                           f"has name \"{df['name']}__\", "\
                                           f"has alias $a via $i1, "\
                                           f"has description $a via $i2; "\
                                           f"delete $i1, $i2;"
            update_flg = False
            first_attr_flg = True
            graql_attribute_update_query = f"match $i isa Vulnerability , " \
                                           f"has name \"{df['name']}__\"; insert $i"
            if df['description'] != 'NA':
                # graql_attribute_delete_query += f" has description $a via $i2; "\
                graql_attribute_update_query += f"  has description \"{df['description']}\" "
                if (first_attr_flg) & (len(aliases)>0):
                    graql_attribute_update_query += ","
                update_flg = True
                first_attr_flg = False
            print(len(aliases))
            if len(aliases) > 0:
                s = "|"
                graql_attribute_update_query += f" has alias \"{s.join(aliases).replace(' ','')}\" " 
                if first_attr_flg:
                     graql_attribute_update_query += ","
                update_flg = True
                first_attr_flg = False
            graql_attribute_update_query += ";"
            print(graql_attribute_update_query)
            if update_flg:
                with session.transaction().write() as write_transaction:
                    # make a write transection with the query
                    write_transaction.query(graql_attribute_delete_query)
                    write_transaction.query(graql_attribute_update_query)
                    # remember to commit at the end
                    write_transaction.commit()           
        else:
            # print("Insert intrusion-set...")
            print("Insert: " + df['name'].strip())
            graql_insert_query = f"insert $x isa Vulnerability, " \
                                 f"has internal_id_key \"{df['internal_id_key']}\", " \
                                 f"has stix_id_key \"{df['stix_id_key']}\", " \
                                 f"has stix_label \"{df['stix_label']}\", " \
                                 f"has entity_type \"{df['entity_type']}\", " \
                                 f"has created {df['created']}, " \
                                 f"has revoked \"{df['revoked']}\", "\
                                 f"has created_at {df['created_at']}, " \
                                 f"has created_at_day \"{df['created_at_day']}\", " \
                                 f"has created_at_month \"{df['created_at_month']}\", " \
                                 f"has created_at_year \"{df['created_at_year']}\", "\
                                 f"has updated_at {df['updated_at']}, "\
                                 f"has name \"{df['name'].strip()}__\" "
                                 
            if df['description'] != 'NA':
                graql_insert_query += f", has description \"{df['description']}\" "
            if len(aliases) > 0:
                s = "|"
                graql_insert_query += f", has alias \"{s.join(aliases)}\" "
            #if f is not None:
            #    graql_insert_query += f", has first_seen {f} "
            #if f_d is not None:
            #    graql_insert_query += f", has first_seen_day \"{f_d}\" "
            #if f_m is not None:
             #   graql_insert_query += f", has first_seen_month \"{f_m}\" "
            #if f_m is not None:
            #    graql_insert_query += f", has first_seen_year \"{df['first_seen_year']}\""
            graql_insert_query += ";"
            print(graql_insert_query)
            with session.transaction().write() as write_transaction:
                # make a write transection with the query
                write_transaction.query(graql_insert_query)
                # remember to commit at the end
                write_transaction.commit()

In [130]:
def load_data_into_grakn(session,input_df):
    print("Vulnerability...")
    # using progress_apply instead of apply so we have a progress bar form tqdm
    # input_df.progress_apply(insert_one_intrusion_set, axis=1, session=session)
    print("Vulnerability...")
    for index,row in input_df.iterrows():
        insert_one_vulnerability(df=row,session=session)

In [131]:
def build_grakn_graph(input_df, keyspace_name):
    with GraknClient(uri="192.168.1.250:48555") as client:
        with client.session(keyspace = keyspace_name) as session:
            load_data_into_grakn(session,input_df)

In [132]:
build_grakn_graph(input_df=result,keyspace_name='grakn')

Vulnerability...
Vulnerability...
Insert: CVE-2009-4324
insert $x isa Vulnerability, has internal_id_key "9b996929-d362-45f8-b239-77e25f4337a3", has stix_id_key "Vulnerability--c09e1718-7d91-4637-8050-c06797a89723", has stix_label "NA", has entity_type "Vulnerability", has created 2020-01-06T15:39:04, has revoked "False", has created_at 2020-01-06T15:39:04, has created_at_day "2020-01-06", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-06T15:39:04, has name "CVE-2009-4324__" , has description "Use-after-free vulnerability in the Doc.media.newPlayer method in Multimedia.api in Adobe Reader and Acrobat 9.x before 9.3, and 8.x before 8.2 on Windows and Mac OS X, allows remote attackers to execute arbitrary code via a crafted PDF file using ZLib compressed streams, as exploited in the wild in December 2009." ;
Insert: CVE-2011-0611
insert $x isa Vulnerability, has internal_id_key "c0ea9b54-75d5-4aed-9364-67fde1de4e71", has stix_id_key "Vulnerability--e54

insert $x isa Vulnerability, has internal_id_key "8dc8b25c-c087-4c50-8fcf-cfff50dfd4a5", has stix_id_key "Vulnerability--1a0f15b9-3046-451c-856a-b10e3da5a80c", has stix_label "NA", has entity_type "Vulnerability", has created 2020-01-06T15:39:04, has revoked "False", has created_at 2020-01-06T15:39:04, has created_at_day "2020-01-06", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-06T15:39:04, has name "CVE-2016-0984__" , has description "Use-after-free vulnerability in Adobe Flash Player before 18.0.0.329 and 19.x and 20.x before 20.0.0.306 on Windows and OS X and before 11.2.202.569 on Linux, Adobe AIR before 20.0.0.260, Adobe AIR SDK before 20.0.0.260, and Adobe AIR SDK & Compiler before 20.0.0.260 allows attackers to execute arbitrary code via unspecified vectors, a different vulnerability than CVE-2016-0973, CVE-2016-0974, CVE-2016-0975, CVE-2016-0982, and CVE-2016-0983." ;
Insert: CVE-2016-4117
insert $x isa Vulnerability, has internal_id_key "

In [122]:
result['description'][5].replace('\"','')

'\nThe (1) ListView, (2) ListView2, (3) TreeView, and (4) TreeView2 ActiveX controls in MSCOMCTL.OCX in the Common Controls in Microsoft Office 2003 SP3, 2007 SP2 and SP3, and 2010 Gold and SP1; Office 2003 Web Components SP3; SQL Server 2000 SP4, 2005 SP4, and 2008 SP2, SP3, and R2; BizTalk Server 2002 SP1; Commerce Server 2002 SP4, 2007 SP2, and 2009 Gold and R2; Visual FoxPro 8.0 SP1 and 9.0 SP2; and Visual Basic 6.0 Runtime allow remote attackers to execute arbitrary code via a crafted (a) web site, (b) Office document, or (c) .rtf file that triggers system state corruption, as exploited in the wild in April 2012, aka MSCOMCTL.OCX RCE Vulnerability.'